In [1]:
import wptherml
from matplotlib import pyplot as plt
import numpy as np



peyton_args = {
     "Number_of_Excitons": 1,
     "number_of_boson_levels": 2,
     "boson_energy_ev": 0.5 / 3.6749322175665e-2, #0375 / 3.6749322175665e-2,
     "exciton_energy_ev" : 0.5 / 3.6749322175665e-2, #0375 / 3.6749322175665e-2,
     "exciton_boson_coupling_ev" : 0.02 / 3.6749322175665e-2,
     "boson_spontaneous_emission_rate_mev" : 0.0, #0.15e3,
     "exciton_spontaneous_emission_rate_mev" : 0.,
     "exciton_dephasing_rate_mev" : 0.,
     "time_step_au" : 1.0
    
}

sf = wptherml.SpectrumFactory()

# instantiate cases
test_1 = sf.spectrum_factory("Spin-Boson", peyton_args)
print("offset")
print(test_1.energy_eigenvalues - 0.25)

Energy Eigenvalues in atomic units are
[0.25 0.73 0.77 1.25]
Energy eigenvalues in eV are
[ 6.80284656 19.86431196 20.95276741 34.01423281]
offset
[0.   0.48 0.52 1.  ]


Now that we have built the Hamiltonian and diagonalized it, we will do some basic checks of the solutions.
In particular, we will print out the eigenvectors for the first few states, and we will test to make sure that the bra-ket with the Hamiltonian and a particular eigenstate is exactly equal to the corresponding eigenvalue of that state, e.g.

$$ \langle \psi_3 | \hat{H} | \psi_3\rangle = E_3 $$

In [2]:
# set the spin spontaneous emission rate consisten with Peytons of 5e-7
test_1.exciton_spontaneous_emission_rate_au = 5e-6

# set the spin dephasing rate consisten with Peytons 1e-9
#test_1.exciton_dephasing_rate_au = 1e-9

In [3]:
# print ground state eigenvector
print(test_1.energy_eigenvectors[:,0])
# print first excited state eigenvector
print(test_1.energy_eigenvectors[:,1])
# print second excited state eigenvector
print(test_1.energy_eigenvectors[:,2])
# print third excited state eigenvector
print(test_1.energy_eigenvectors[:,3])
    

# compute expectation value of energy of third excited state
# get ket and bra of \psi_3
ket = test_1.energy_eigenvectors[:,3].reshape(-1,1)
bra = np.conj(ket).T

# store Hamiltonian 
HP = test_1.hamiltonian_matrix

# compute <\psi_3 | H | \psi_3>
exp_psi3 = np.dot(bra, np.dot(HP, ket))

# check that it is thesame as eigenvalue of third excited state
print("Checking to see if <\psi_3 | H | \psi_3> is the same as the eigenvalue of the third excited state")
print(np.isclose(exp_psi3[0,0], test_1.energy_eigenvalues[3]))


[1. 0. 0. 0.]
[ 0.         -0.70710678  0.70710678  0.        ]
[0.         0.70710678 0.70710678 0.        ]
[0. 0. 0. 1.]
Checking to see if <\psi_3 | H | \psi_3> is the same as the eigenvalue of the third excited state
True


Now we will initialize the state as $|gge\rangle$ where the cavity is in the ground state, the first spin is in the ground state, and the second spin is in the excited state.  We will compute the dynamics and store the populations after 1000 time stetps.

In [11]:
initial_cav = np.array([[1],[0]])
initial_sp1 = np.array([[1],[0]])
initial_sp2 = np.array([[0],[1]])

# construct initial composite state
#initial_ket = np.kron(np.kron(initial_cav, initial_sp1), initial_sp2)
initial_ket = np.kron(initial_cav, initial_sp2)
print("Initial state")
print(initial_ket)

# compute initial density matrix of composite system
initial_rho = np.dot(initial_ket, np.conj(initial_ket).T)

# print initial rho
print("Initial density matrix")
print(initial_rho)

Initial state
[[0]
 [1]
 [0]
 [0]]
Initial density matrix
[[0 0 0 0]
 [0 1 0 0]
 [0 0 0 0]
 [0 0 0 0]]


In [12]:
# set initial density matrix
test_1.rho = initial_rho

for i in range(10):
    test_1.rk4_update_on_rho()
    print(F"T= {i+1} Rho")
    print(test_1.rho)


print("Final density matrix")
print(test_1.rho)
    

T= 1 Rho
[[4.99932084e-06+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         9.99595055e-01+0.j
  0.00000000e+00+0.01999459j 0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00-0.01999459j
  3.99945667e-04+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]]
T= 2 Rho
[[9.99461830e-06+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         9.98390867e-01+0.j
  0.00000000e+00+0.03995705j 0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00-0.03995705j
  1.59913881e-03+0.j         0.00000000e+00+0.j        ]
 [0.00000000e+00+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]]
T= 3 Rho
[[1.49819005e-05+0.j         0.00000000e+00+0.j
  0.00000000e+00+0.j         0.00000000e+00+0.j        ]
 [0.000000

In [ ]:
# create new dictionary to store calculation data
TC_data = {

    "spin_1_initial_state" : [1, 0],
    "spin_2_initial_state" : [0, 1],
    "cavity_initial_state" : [1, 0],
    "spin_frequency" : 0.5,
    "cavity_frequency" : 0.5,
    "cavity_coupling" : 0.02,
    "cavity_spontaneous_emission" : 0,
    "spin_spontaneous_emission" : 5e-7,
    "cavity_dephasing" : 0,
    "spin_dephasing" : 1e-9,
    "time_steps" : 1000,
    "time_step_size" : 1.0,
    "population_s1g_s2g_cg" : [],
    "population_s1e_s2g_cg" : [],
    "population_s1g_s2e_cg" : [],
    "population_s1e_s2e_cg" : [],
    "population_s1g_s2g_ce" : [],
    "population_s1e_s2g_ce" : [],
    "population_s1g_s2e_ce" : [],
    "population_s1e_s2e_ce" : [],
}



# run dynamics and store the results in the dictionary
TC_data["population_s1g_s2g_cg"] = np.real(test_1.rho[0,0])
TC_data["population_s1g_s2e_cg"] = np.real(test_1.rho[1,1])
TC_data["population_s1e_s2g_cg"] = np.real(test_1.rho[2,2])
TC_data["population_s1e_s2e_cg"] = np.real(test_1.rho[3,3])
TC_data["population_s1g_s2g_ce"] = np.real(test_1.rho[4,4])
TC_data["population_s1g_s2e_ce"] = np.real(test_1.rho[5,5])
TC_data["population_s1e_s2g_ce"] = np.real(test_1.rho[6,6])
TC_data["population_s1e_s2e_ce"] = np.real(test_1.rho[7,7])

In [ ]:
import json
# write the data to a JSON file
def write_to_json(data, filename):
    with open(filename, 'w') as json_file:
        json.dump(data, json_file, indent=4)

def create_output_filename(dictionary):
    # define output file name based on the parameters of the simulation
    output_filename = "TC_simulation_"
    output_filename += "spin_freq_" + str(dictionary["spin_frequency"]) + "_"
    output_filename += "cavity_freq_" + str(dictionary["cavity_frequency"]) + "_"
    output_filename += "cavity_coupling_" + str(dictionary["cavity_coupling"]) + "_"
    output_filename += "cavity_spontaneous_emission_" + str(dictionary["cavity_spontaneous_emission"]) + "_"
    output_filename += "spin_spontaneous_emission_" + str(dictionary["spin_spontaneous_emission"]) + "_"
    output_filename += "cavity_dephasing_" + str(dictionary["cavity_dephasing"]) + "_"
    output_filename += "spin_dephasing_" + str(dictionary["spin_dephasing"]) + ".json"
    return output_filename

output_filename = create_output_filename(TC_data)

write_to_json(TC_data, output_filename)

In [ ]:
print(np.trace(test_1.rho))

In [ ]:
p = 0.4017711449000221 + 0.11334977116080829 + 0.13513205050371 + 0.3497470334354594
print(p)